
This project is aim at finding within-text data citations in scientific research articles. The results from this project can show how public data are being used in science and help the policy makers make wiser, more transparent public investments. It will help move researchers and governments from using ad-hoc methods to automated ways of finding out what datasets are being used to solve problems, what measures are being generated, and which researchers are the experts

In [ ]:
# from zipfile import ZipFile as zp
import re
# import os
# from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
# import nltk
# import spacy
# from nltk.tokenize import sent_tokenize, word_tokenize
# from unidecode import unidecode
import numpy as np
# from urllib.request import Request, urlopen
# import requests
# import bs4
import pandas as pd
# import matplotlib.pyplot as plt
from tqdm import tqdm
# import json
# import itertools
# from gensim.models import Word2Vec
# from nltk.corpus import wordnet as wn
# from numpy import array
from imblearn.over_sampling import SMOTE

from keras.preprocessing.text import Tokenizer
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout

# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# from sklearn.linear_model import LogisticRegression

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words = set(list(stop_words))
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
let_me_tize_u = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /cluster/home/coguik01/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /cluster/home/coguik01/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Unzip Training Files

In [ ]:
with zp('/cluster/tufts/data/train.csv.zip') as zipObj:
    zipObj.extractall('/cluster/tufts//data')

#### Retrieve Data Rubric

In [ ]:
# Load Data Rubric
train_pd = pd.read_csv('/cluster/tufts/data/train.csv')
train_pd.drop_duplicates(subset ="Id", keep = 'first', inplace = True)# there are duplicate publications in the dataset
train_pd = train_pd.reset_index(drop=True)
train_pd

,Id,pub_title,dataset_title,dataset_label,cleaned_label
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
...,...,...,...,...,...
14311,f89dd9fa-07af-4384-aa0c-0d14602c0cea,Artificial Intelligence of COVID-19 Imaging: A...,RSNA International COVID-19 Open Radiology Dat...,RSNA International COVID-19 Open Radiology Dat...,rsna international covid 19 open radiology dat...
14312,b3498176-8832-4033-aea6-b5ea85ea04c4,RSNA International Trends: A Global Perspectiv...,RSNA International COVID-19 Open Radiology Dat...,RSNA International COVID Open Radiology Database,rsna international covid open radiology database
14313,f77eb51f-c3ac-420b-9586-cb187849c321,MCCS: a novel recognition pattern-based method...,CAS COVID-19 antiviral candidate compounds dat...,CAS COVID-19 antiviral candidate compounds dat...,cas covid 19 antiviral candidate compounds dat...
14314,ab59bcdd-7b7c-4107-93f5-0ccaf749236c,Quantitative Structure–Activity Relationship M...,CAS COVID-19 antiviral candidate compounds dat...,CAS COVID-19 antiviral candidate compounds dat...,cas covid 19 antiviral candidate compounds dat...


In [ ]:
s = pd.read_csv('/cluster/tufts/data/sample_submission.csv')
s

,Id,PredictionString
0,2100032a-7c33-4bff-97ef-690822c43466,NaN
1,2f392438-e215-4169-bebf-21ac4ff253e1,NaN
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,NaN
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,NaN


#### Peek at Data

In [ ]:
#how to peek into a section in an article in the data
f = open('/cluster/tufts/data/train/' + train_pd['Id'][0] + '.json',)

# returns JSON object as
# a dictionary
data = json.load(f)
print(data[0]) #this is the text in the first section of the first article

#### Merge text from all sections in the data

In [ ]:
# Merge all the texts in different sections into one full chunk for each article
content = []
sub_content = []
for i in tqdm(range(0,len(train_pd))):
    f = open('/cluster/tufts/data/train/' + train_pd['Id'][i] + '.json',)
    data = json.load(f)
    d = ''
    for j in range(0,len(data)):
        d = d + data[j]['text']+" "
    content.append(d)



100%|██████████| 14316/14316 [04:15<00:00, 56.12it/s]


In [ ]:
# Transfer articles into a dataframe so it can be pickled
articles = pd.DataFrame(data=[content])
articles = articles.T
articles.columns = ["Raw Text"]
articles.to_pickle('/cluster/tufts/data/Raw_text_data_2.pkl')
articles

,Raw Text
0,This study used data from the National Educati...
1,Dropping out of high school is not necessarily...
2,", stress satisfactory outcomes for all youth,..."
3,Federal Reserve Bank of Richmond S1. Accountin...
4,This article investigates an important factor ...
...,...
14311,T he coronavirus disease 2019 (COVID-19) pande...
14312,Our lives have been fundamentally altered this...
14313,"The outbreak of the coronavirus disease 2019 ,..."
14314,The ongoing COVID-19 pandemic has challenged t...


In [ ]:
articles = pd.read_pickle('/cluster/tufts/data/Raw_text_data.pkl')

#### Break down articles in to sentences

In [ ]:
#break articles down into sentence tokens
sent_tokens = []

for i in tqdm(range(0,len(articles))):
    sent_tokens.append(sent_tokenize(articles['Raw Text'][i]))

sent_token_articles = pd.DataFrame(data=[sent_tokens])
sent_token_articles = sent_token_articles.T
sent_token_articles.columns = ["Sentence Tokenized Text"]
sent_token_articles.to_pickle('/cluster/tufts/data/sent_tokenized_text_data.pkl')

In [ ]:
#sent_token_articles = pd.read_pickle('/cluster/tufts/data/sent_tokenized_text_data.pkl')
sent_token_articles['label'] = train_pd['dataset_label'].to_list()
sent_token_articles

,Sentence Tokenized Text,label
0,[This study used data from the National Educat...,National Education Longitudinal Study
1,[Dropping out of high school is not necessaril...,National Education Longitudinal Study
2,"[ , stress satisfactory outcomes for all youth...",National Education Longitudinal Study
3,"[Federal Reserve Bank of Richmond S1., Account...",National Education Longitudinal Study
4,[This article investigates an important factor...,National Education Longitudinal Study
...,...,...
14311,[T he coronavirus disease 2019 (COVID-19) pand...,RSNA International COVID-19 Open Radiology Dat...
14312,[Our lives have been fundamentally altered thi...,RSNA International COVID Open Radiology Database
14313,[The outbreak of the coronavirus disease 2019 ...,CAS COVID-19 antiviral candidate compounds dat...
14314,[The ongoing COVID-19 pandemic has challenged ...,CAS COVID-19 antiviral candidate compounds dat...


In [ ]:
clean Sentence Tokenized

#### Get the sentences that have the referenced data

In [ ]:
# Note: more than one sentence may have the referenced article in each paper (i.e the data may be referenced more than once)
main_list = []
sub_list = []
for i in tqdm(range(0,len(sent_token_articles))):
    for j in range(0,len(sent_token_articles['Sentence Tokenized Text'][i])):
        if (sent_token_articles['Sentence Tokenized Text'][i][j].find(train_pd['cleaned_label'][i]) != -1):
            sub_list.append(sent_token_articles['Sentence Tokenized Text'][i][j])
    main_list.append(sub_list)
    sub_list = []

100%|██████████| 14316/14316 [00:37<00:00, 379.78it/s]


### Second method of extracting verbs

In [ ]:
# Note: more than one sentence may have the referenced article in each paper (i.e the data may be referenced more than once)
main_list = []
sub_list = []
isword_in_label = True
for i in tqdm(range(0,len(sent_token_articles))):
    label_text = train_pd['cleaned_label'][i].split(' ')
    for j in range(0,len(sent_token_articles['Sentence Tokenized Text'][i])):
        for k in range(0,len(label_text)):
            isword_in_label = (isword_in_label and sent_token_articles['Sentence Tokenized Text'][i][j].find(label_text[k]) != -1)
        if isword_in_label == True:
            sub_list.append(sent_token_articles['Sentence Tokenized Text'][i][j])
        isword_in_label = True
    main_list.append(sub_list)
    sub_list = []

100%|██████████| 14316/14316 [00:23<00:00, 606.42it/s]


In [ ]:
temp = []
isword_in_label = True
label_text = train_pd['cleaned_label'][1].split(' ')
for j in range(0,len(sent_token_articles['Sentence Tokenized Text'][1])):
    for k in range(0,len(label_text)):
        isword_in_label = (isword_in_label and sent_token_articles['Sentence Tokenized Text'][1][j].find(label_text[k]) != -1)
    if isword_in_label == True:
        temp.append(sent_token_articles['Sentence Tokenized Text'][1][j])
    isword_in_label = True

In [ ]:
temp

['using data on public and private school students from the national education longitudinal study of 1988 (nels:88), berktold, geis, and kaufman (1998) examined the educational attainment of the 21 percent of 1988 eighth graders who had dropped out of high school at least once between eighth grade and the spring of 1994, 2 years after they would have graduated if they had finished with the majority of their cohort.',
 'source: u.s. department of education, national center for education statistics, national education longitudinal study of 1988 (nels:88/2000).']

In [ ]:
isword_in_label = True
isword_in_label and sent_token_articles['Sentence Tokenized Text'][1][1].find(label_text[2]) != -1

False

In [ ]:
sent_token_articles['Sentence Tokenized Text'][1][15]

'source: u.s. department of education, national center for education statistics, national education longitudinal study of 1988 (nels:88/2000).'

In [ ]:
# sents_w_labels = pd.DataFrame(data=[main_list])
# sents_w_labels = sents_w_labels.T
# sents_w_labels.columns = ["Sentences With Labels"]
# sents_w_labels
sents_w_labels.to_pickle('/cluster/tufts/data/sents_with_labels.pkl')

In [ ]:
sents_w_labels  = pd.read_pickle('/cluster/tufts/data/sents_with_labels.pkl')
sents_w_labels["Sentences With Labels"][0]

['This study used data from the National Education Longitudinal Study (NELS:88) to examine the effects of dual enrollment programs for high school students on college degree attainment.',
 'Any college degree attainment The study author collected information on college degree attainment from the fourth follow-up of the National Education Longitudinal Study collected in 2000.',
 "The study author collected information on bachelor's degree attainment from the fourth follow-up of the National Education Longitudinal Study collected in 2000."]

In [ ]:
main_l = []
id_ = []
for i in tqdm(range(0,len(sent_data))):
    for j in range(0,len(sent_data['Tokenized Articles'][i])):
        if (sent_data['Tokenized Articles'][i][j].find(train_pd['dataset_label'][i]) != -1):
            main_l.append(sent_data['Tokenized Articles'][i][j])
            id_.append(i)
            break

100%|██████████| 19661/19661 [00:06<00:00, 3064.03it/s]


In [ ]:
len(main_l)

4120

In [ ]:
#get list of unique labels in the dataset
dd = train_pd['dataset_label'].to_list()
dd = list(np.unique(dd))
dd

#### create bow for verbs

In [ ]:
#Load sentences with labels
sents_w_labels = pd.read_pickle('/cluster/tufts/data/sents_with_labels.pkl')
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Go through all the sentences with labels and tag POS
main_list = []
sub_list = []
for i in tqdm(range(0,len(sents_w_labels))):
    for j in range(0,len(sents_w_labels['Sentences With Labels'][i])):
        dd = nlp(sents_w_labels['Sentences With Labels'][i][j])
        sub_list.append(dd)

    main_list.append(sub_list)
    sub_list = []

100%|██████████| 19661/19661 [08:19<00:00, 39.40it/s] 


In [ ]:
# save the parts of speech labelled sentences
sents_w_labels_spacy = pd.DataFrame(data=[main_list])
sents_w_labels_spacy = sents_w_labels_spacy.T
sents_w_labels_spacy.columns = ["POS Tagged Sentences With Labels"]
sents_w_labels_spacy.to_pickle('/cluster/tufts/data/sents_with_labels_spacy.pkl')

In [ ]:
# Go through all the sentences with labels and extract the verbs
main_list = []
sub_list = []
for i in tqdm(range(0,len(sents_w_labels_spacy))):
    for j in range(0,len(sents_w_labels_spacy["POS Tagged Sentences With Labels"][i])):
        dd = [let_me_tize_u.lemmatize(token.text,pos='v').lower() for token in sents_w_labels_spacy["POS Tagged Sentences With Labels"][i][j] if token.pos_ == 'VERB']
        sub_list.append(dd)

    main_list.append(sub_list)
    sub_list = []

100%|██████████| 19661/19661 [00:00<00:00, 19976.59it/s]


In [ ]:
# Go through all the verbs and re-lemmatize (I didn't get good lemmatization in the previous loop)
main_list = []
sub_list = []
for i in tqdm(range(0,len(sents_w_labels_spacy_verbs))):
    for j in range(0,len(sents_w_labels_spacy_verbs["Verbs in Sentences With Labels"][i])):
        dd = [let_me_tize_u.lemmatize(token,pos='v') for token in sents_w_labels_spacy_verbs["Verbs in Sentences With Labels"][i][j]]
        sub_list.append(dd)

    main_list.append(sub_list)
    sub_list = []

In [ ]:
# save the parts of speech labelled sentences
sents_w_labels_spacy_verbs = pd.DataFrame(data=[main_list])
sents_w_labels_spacy_verbs = sents_w_labels_spacy_verbs.T
sents_w_labels_spacy_verbs.columns = ["Verbs in Sentences With Labels"]
sents_w_labels_spacy_verbs.to_pickle('/cluster/tufts/data/sents_with_labels_spacy_verbs.pkl')

In [ ]:
#sents_w_labels_spacy_verbs = pd.read_pickle('/cluster/tufts/data/sents_with_labels_spacy_verbs.pkl')
sent_token_articles = pd.read_pickle('/cluster/tufts/data/sent_tokenized_text_data.pkl')

In [ ]:
# create list of unique verbs for each sentence with a label
main_list = []

for i in range(0,len(sents_w_labels_spacy_verbs)):
    new_list = []
    for j in range(0,len(sents_w_labels_spacy_verbs["Verbs in Sentences With Labels"][i])):
        new_list = new_list + sents_w_labels_spacy_verbs["Verbs in Sentences With Labels"][i][j]
    final_list = list(np.unique(new_list))
    main_list.append(final_list)



### An Example of the quality of verbs discovered by Spacy

In [ ]:
sents_w_labels_spacy_verbs = pd.read_pickle('/cluster/tufts/data/sents_with_labels_spacy_verbs.pkl')

main_list = []

for i in range(0,len(sents_w_labels_spacy_verbs)):
    new_list = []
    for j in range(0,len(sents_w_labels_spacy_verbs["Verbs in Sentences With Labels"][i])):
        new_list = new_list + sents_w_labels_spacy_verbs["Verbs in Sentences With Labels"][i][j]
    final_list = list(np.unique(new_list))
    main_list.append(final_list)


sents_v = pd.DataFrame(data=[main_list])
sents_v= sents_v.T
sents_v.columns = ["Verbs"]

convert_v_list = sents_v['Verbs'].to_list()

v_list = []
for i in range(0,len(convert_v_list)):
    v_list = v_list + convert_v_list[i]
unique_v_list = list(np.unique(v_list))

occurrences = []
for i in range(0,len(unique_v_list)):
    occurrences.append(v_list.count(unique_v_list[i]))

bow_df = pd.DataFrame(columns=['Verbs','Occurrences'])
bow_df['Verbs'] = unique_v_list
bow_df['Occurrences'] = occurrences
bow_df.head(50)

,Verbs,Occurrences
0,#,1
1,'d,1
2,):,1
3,-,86
4,-(c,1
5,-------*----g------------g--------------------...,1
6,-.-----v-----d--------d,1
7,-136,1
8,-17,1
9,-50,1


In [ ]:
# create data frame with sentences from all articles in rows
main_list = []
labels = []

for i in tqdm(range(0,len(sent_token_articles))):
    for j in range(0,len(sent_token_articles['Sentence Tokenized Text'][i])):
        main_list.append(sent_token_articles['Sentence Tokenized Text'][i][j])
        if (sent_token_articles['Sentence Tokenized Text'][i][j].find(sent_token_articles['label'][i]) != -1):
            labels.append(1)
        else:
            labels.append(0)



100%|██████████| 14316/14316 [00:50<00:00, 284.11it/s]


In [ ]:
len(labels)

4178921

In [ ]:
all_sents = pd.DataFrame(data=[main_list])
all_sents = all_sents.T
all_sents.columns = ["Sentences"]
all_sents['labels'] = labels
all_sents

,Sentences,labels
0,This study used data from the National Educati...,1
1,The study also reported whether the impacts of...,0
2,"In addition, a supplemental analysis reports o...",0
3,Dual enrollment programs offer college-level l...,0
4,The programs offer college courses and/or the ...,0
...,...,...
4178916,As a consequence of the current COVID-19 pande...,0
4178917,"As a side effect of analyzing the data, we are...",0
4178918,"In addition, the workflow has been used to det...",0
4178919,The material has been used successfully for te...,0


In [ ]:
all_sents.to_pickle('/cluster/tufts/data/all_sents.pkl')
#sents_v.to_pickle('/cluster/tufts/data/sents_v.pkl')

In [ ]:
df.to_pickle('/cluster/tufts/data/sents_with_data.pkl')

### New method of getting the verbs with wordnet

In [ ]:
#load sentences with labels

sents_w_labels  = pd.read_pickle('/cluster/tufts/data/sents_with_labels.pkl')
sents_w_labels

,Sentences With Labels
0,[this study used data from the national educat...
1,[using data on public and private school stude...
2,"[using the nationally representative, longitud..."
3,[national education longitudinal study of 1988...
4,[using data from the national education longit...
...,...
14311,[the rsna has developed the medical imaging an...
14312,[the international community came together to ...
14313,"[then, several 3cl pro ligand complexes are u..."
14314,[we used the resulting models to screen 1087 f...


In [ ]:
# Check if all sentences with labels were gotten
empty = []
for i in range(0,len(sents_w_labels)):
    if len(sents_w_labels['Sentences With Labels'][i]) == 0:
        empty.append(i)


In [ ]:
# 13188th data label is different from cleaned_label: shorter
# 13755th data not referenced in the same manner as label (acually very different)
# 13924th data not referenced in the same manner as label (acually very different)
empty

[]

In [ ]:
target = 'covid 19 open research dataset'
for i in range(0,len(sent_token_articles['Sentence Tokenized Text'][13188])):
    if sent_token_articles['Sentence Tokenized Text'][13188][i].find(target) != -1:
        sentence = sent_token_articles['Sentence Tokenized Text'][13188][i]
        #break

In [ ]:
sent_token_articles = pd.read_pickle('/cluster/tufts/data/sent_tokenized_text_data.pkl')
sent_token_articles

,Sentence Tokenized Text,label
0,[This study used data from the National Educat...,National Education Longitudinal Study
1,[Dropping out of high school is not necessaril...,National Education Longitudinal Study
2,"[ , stress satisfactory outcomes for all youth...",National Education Longitudinal Study
3,"[Federal Reserve Bank of Richmond S1., Account...",National Education Longitudinal Study
4,[This article investigates an important factor...,National Education Longitudinal Study
...,...,...
14311,[T he coronavirus disease 2019 (COVID-19) pand...,RSNA International COVID-19 Open Radiology Dat...
14312,[Our lives have been fundamentally altered thi...,RSNA International COVID Open Radiology Database
14313,[The outbreak of the coronavirus disease 2019 ...,CAS COVID-19 antiviral candidate compounds dat...
14314,[The ongoing COVID-19 pandemic has challenged ...,CAS COVID-19 antiviral candidate compounds dat...


In [ ]:
# a little preprocessing
sub_list = []
for i in tqdm(range(0,len(sent_token_articles))):
    for j in range(0,len(sent_token_articles['Sentence Tokenized Text'][i])):
        content = sent_token_articles['Sentence Tokenized Text'][i][j].lower()
        content = content.replace('-', ' ')
        content = content.replace('(', ' ')
        content = content.replace(')', ' ')
        sub_list.append(content)


    sent_token_articles['Sentence Tokenized Text'][i] = sub_list
    sub_list = []

100%|██████████| 14316/14316 [00:00<00:00, 42007.80it/s]


In [ ]:
count

4178921

In [ ]:
word_token_sents = pd.DataFrame(data=[main_list])
word_token_sents = word_token_sents.T
word_token_sents.columns = ["Word Tokenized Sentences With Labels"]
#word_token_sents['labels'] = labels
word_token_sents
word_token_sents.to_pickle('/cluster/tufts/data/word_token_sents_with_labels.pkl')

In [ ]:
word_token_sents = pd.read_pickle('/cluster/tufts/data/word_token_sents_with_labels.pkl')
word_token_sents

,Word Tokenized Sentences With Labels
0,"[[this, study, used, data, from, the, national..."
1,"[[using, data, public, and, private, school, s..."
2,"[[using, the, nationally, representative, long..."
3,"[[national, education, longitudinal, study, 19..."
4,"[[using, data, from, the, national, education,..."
...,...
14311,"[[the, rsna, has, developed, the, medical, ima..."
14312,"[[the, international, community, came, togethe..."
14313,"[[then, several, 3cl, pro, ligand, complexes, ..."
14314,"[[used, the, resulting, models, screen, 1087, ..."


### use newest method to extract verbs

In [ ]:
sent_token_articles = pd.read_pickle('/cluster/tufts/data/sent_tokenized_text_data_RTdatasetRT_combined.pkl')

# Note: more than one sentence may have the referenced article in each paper (i.e the data may be referenced more than once)
replacement_term = 'RTdatasetRT'
main_list = []
sub_list = []
for i in tqdm(range(0,len(sent_token_articles))):
    for j in range(0,len(sent_token_articles['Sentence Tokenized Text'][i])):
        if (sent_token_articles['Sentence Tokenized Text'][i][j].find(replacement_term) != -1):
            sub_list.append(sent_token_articles['Sentence Tokenized Text'][i][j])
    main_list.append(sub_list)
    sub_list = []

100%|██████████| 14316/14316 [00:19<00:00, 730.63it/s] 


In [ ]:
sents_w_labels = pd.DataFrame(data=[main_list])
sents_w_labels = sents_w_labels.T
sents_w_labels.columns = ["Sentences With Labels"]
sents_w_labels
sents_w_labels.to_pickle('/cluster/tufts/data/sents_with_labels_bert.pkl')

In [ ]:
sents_w_labels = pd.read_pickle('/cluster/tufts/data/sents_with_labels_bert.pkl')

main_list = []
sub_list = []

for i in tqdm(range(0,len(sents_w_labels))):
    for j in range(0,len(sents_w_labels["Sentences With Labels"][i])):
        sub_list.append(word_tokenize(sents_w_labels["Sentences With Labels"][i][j]))

    main_list.append(sub_list)
    sub_list = []

100%|██████████| 14316/14316 [00:11<00:00, 1204.40it/s]


In [ ]:
word_token_sents = pd.DataFrame(data=[main_list])
word_token_sents = word_token_sents.T
word_token_sents.columns = ["Word Tokenized Sentences With Labels"]
#word_token_sents['labels'] = labels
#word_token_sents
word_token_sents.to_pickle('/cluster/tufts/data/word_token_sents_with_labels_bert.pkl')

In [ ]:
# Extract verbs using nltk's wordnet
verb_list = []
sub_list = []
trivial_verbs = ['be','have']

for i in tqdm(range(0,len(word_token_sents))):
    for j in range(0,len(word_token_sents["Word Tokenized Sentences With Labels"][i])):
        for k in range(0,len(word_token_sents["Word Tokenized Sentences With Labels"][i][j])):
            if word_token_sents["Word Tokenized Sentences With Labels"][i][j][k] == 'rtdatasetrt':
                sub_list.append(word_token_sents["Word Tokenized Sentences With Labels"][i][j][k])
            else:
                lemma = let_me_tize_u.lemmatize(word_token_sents["Word Tokenized Sentences With Labels"][i][j][k],pos='v')
                if lemma not in trivial_verbs:
                    verb_sets = []
                    wrd = wn.synsets(word_token_sents["Word Tokenized Sentences With Labels"][i][j][k], pos=wn.VERB)
                    for item in wrd:
                        verb_sets.append(item.lemmas()[0].name())

                    noun_sets = []
                    wrd = wn.synsets(word_token_sents["Word Tokenized Sentences With Labels"][i][j][k], pos=wn.NOUN)
                    for item in wrd:
                        noun_sets.append(item.lemmas()[0].name())

                    if len(verb_sets) > len(noun_sets):
                        sub_list.append(word_token_sents["Word Tokenized Sentences With Labels"][i][j][k])

    verb_list.append(sub_list)
    sub_list = []

100%|██████████| 14316/14316 [00:57<00:00, 249.57it/s]


In [ ]:
wn.synsets('author',pos=wn.NOUN)
#wrd[0].lemmas()[0].name()

[Synset('writer.n.01'), Synset('generator.n.03')]

In [ ]:
sents_v_wordnet = pd.DataFrame(data=[verb_list])
sents_v_wordnet = sents_v_wordnet.T
sents_v_wordnet.columns = ["Verbs"]
sents_v_wordnet
sents_v_wordnet.to_pickle('/cluster/tufts/data/verbs_sentences_with_labels.pkl')

In [ ]:
sents_v_wordnet = pd.DataFrame(data=[verb_list])
sents_v_wordnet = sents_v_wordnet.T
sents_v_wordnet.columns = ["Verbs"]
sents_v_wordnet.to_pickle('/cluster/tufts/data/verbs_sentences_with_labels_bert.pkl')
sents_v_wordnet

,Verbs
0,"[used, examine, collected, follow, up, collect..."
1,"[using, examined, dropped, out, graduated, fin..."
2,"[using, set, used, examine, associated, partic..."
3,"[follows, presents, requires, disentangle, fol..."
4,"[using, added, includes, include, investigate,..."
...,...
14311,"[developed, attained, buy, contribute]"
14312,"[came, build, further, facilitate]"
14313,"[used, create, characterize, screening, carrie..."
14314,"[used, resulting, approved, drugs, list, assig..."


In [ ]:
# create data frame with sentences from all articles in rows
sent_token_articles = pd.read_pickle('/cluster/tufts/data/sent_tokenized_text_data_RTdatasetRT_combined.pkl')

main_list = []
labels = []

for i in tqdm(range(0,len(sent_token_articles))):
    for j in range(0,len(sent_token_articles['Sentence Tokenized Text'][i])):
        #sentence = sent_token_articles['Sentence Tokenized Text'][i][j]
        main_list.append(word_tokenize(sent_token_articles['Sentence Tokenized Text'][i][j]))
        if  (sent_token_articles['Sentence Tokenized Text'][i][j].find('RTdatasetRT') != -1):
            labels.append(1)
        else:
            labels.append(0)


100%|██████████| 14316/14316 [10:07<00:00, 23.57it/s]  


In [ ]:
all_sents.to_pickle('/cluster/tufts/data/all_sents_BOW_1.pkl')

In [ ]:
all_sents = pd.DataFrame(data=[main_list])
all_sents = all_sents.T
all_sents.columns = ["Sentences"]
all_sents['labels'] = labels
#all_sents
all_sents.to_pickle('/cluster/tufts/data/all_sents_bert.pkl')

In [ ]:
all_sents.to_pickle('/cluster/tufts/data/all_sents.pkl')
#sents_v.to_pickle('/cluster/tufts/data/sents_v.pkl')

In [ ]:
#sent_token_articles = pd.read_pickle('/cluster/tufts/data/sent_tokenized_text_data.pkl')
#a = sent_token_articles.index
b = list(a)[0:10]

In [ ]:
del dataset_df

In [ ]:
# confirm that the number of sentences with data labels match that in the list of sentences
count = 0
for i in tqdm(range(0,len(sents_w_labels))):
    for j in range(0,len(sents_w_labels['Sentences With Labels'][i])):
        count = count + 1

count2 = len(all_sents[all_sents['labels']==1])
if count == count2:
    print('yes')


100%|██████████| 14316/14316 [00:00<00:00, 199871.70it/s]

yes


In [ ]:
# Create data frame to store the unique verbs from the sentences with label and the number of occurrences
# of these verbs in all sentences
sents_v = pd.read_pickle('/cluster/tufts/data/verbs_sentences_with_labels_bert.pkl')
convert_v_list = sents_v['Verbs'].to_list()

v_list = []
for i in range(0,len(convert_v_list)):
    v_list = v_list + convert_v_list[i]
unique_v_list = list(np.unique(v_list))
len(unique_v_list)

occurrences = []
for i in range(0,len(unique_v_list)):
    occurrences.append(v_list.count(unique_v_list[i]))

bow_df = pd.DataFrame(columns=['Verbs','Occurrences'])
bow_df['Verbs'] = unique_v_list
bow_df['Occurrences'] = occurrences
bow_df

,Verbs,Occurrences
0,abandoned,5
1,abate,1
2,abbreviated,8
3,abide,89
4,abort,3
...,...,...
4970,yielded,90
4971,yielding,43
4972,yields,58
4973,zoning,4


In [ ]:
# Create a corpus by filtering out other words that are not verbs seen in sentences with labels

min_occur = 1 # set this value to filter verbs that rare show up in other sentences
all_sents = pd.read_pickle('/cluster/tufts/data/all_sents.pkl')

min_occur_vocab_df = bow_df[bow_df['Occurrences'] >= min_occur]
min_occur_vocab = min_occur_vocab_df['Verbs'].to_list()
min_occur_vocab

# sentences filter by vocab
sentence_list = []
is_empty = []
for i in tqdm(range(0,len(all_sents))):
    tokens = [w for w in all_sents['Sentences'][i] if w in min_occur_vocab]
    sentence_list.append(' '.join(tokens))
    if len(tokens)==0:
        is_empty.append(True)
    else:
        is_empty.append(False)

all_sents['BOW'] = sentence_list
all_sents['Empty'] = is_empty

100%|██████████| 4178921/4178921 [1:40:04<00:00, 696.00it/s]  


In [ ]:
all_sents.to_pickle('/cluster/tufts/data/all_sents_bert_BOW_1.pkl')

In [ ]:
all_sents = pd.read_pickle('/cluster/tufts/data/all_sents_bert_BOW_1.pkl')

### SMOTE

In [ ]:
def create_test_train(all_sents, mode, test_data_size):
    # Remove empty sentences
    dataset_df = all_sents[all_sents['Empty']==False]

    #Get test data
    # get and equal number of sentences with and without labels
    test_df_1 = dataset_df[dataset_df['labels']==1].sample(n=test_data_size)
    test_df_0 = dataset_df[dataset_df['labels']==0].sample(n=test_data_size)

    # Merge two sets of sentences
    test = [test_df_1,test_df_0]
    test_df = pd.concat(test)

    # convert test data to array
    xtest_data = test_df['BOW'].to_list()
    ytest = test_df['labels'].to_list()

    # Remove the the data used in the test set from the data pull (separating test data from train data)
    test_index = list(test_df.index)
    train_df = dataset_df.drop(test_index)

    xtrain_data = train_df['BOW'].to_list()
    ytrain_data = train_df['labels'].to_list()

    #convert test and train data to BOW
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(xtrain_data)

    # encode training data set
    xtrain_bow = tokenizer.texts_to_matrix(xtrain_data, mode=mode)

    #tokenizer.fit_on_texts(xtest_data)
    Xtest = tokenizer.texts_to_matrix(xtest_data, mode=mode)

    #Oversampling the data to create an equal number of classes (sentences with and without data)
    smote = SMOTE(random_state = 101)
    Xtrain, ytrain = smote.fit_resample(xtrain_bow, ytrain_data)

    return Xtrain, ytrain, Xtest, ytest

In [ ]:
# Get the non empty sentences and group them into one with labels (1) and ones without labels (0)
# There are 50,000 sentences with labels and about 3 million sentences without labels so I randomly select 50,000
# without labels to balance the ratio of the two classes
# Another (better) idea is to use SMOTE
train_df_1 = all_sents[all_sents['Empty']==False]
train_df_1_1 = train_df_1[train_df_1['labels']==1]
train_df_1_2 = train_df_1[train_df_1['labels']==0]
first_label_df = train_df_1_1['BOW']
second_label_df = train_df_1_2['BOW'].sample(n=len(train_df_1_1)) # randomly select 50,000 sentences with label 0

In [ ]:
# Creating the corpus takes time so I saved this step
first_label_df = pd.read_pickle('/cluster/tufts/data/first_label_df.pkl')
second_label_df = pd.read_pickle('/cluster/tufts/data/second_label_df.pkl')

In [ ]:
# Function creates testing and training data
def create_test_train(split, mode,first_label_df ,second_label_df):
    test_split = split

    x_first_label = first_label_df.to_list()
    x_second_label = second_label_df.to_list()

    split_index = len(x_first_label)-test_split

    xtrain_data = x_second_label[0:split_index] + x_first_label[0:split_index]
    ytrain = array([0 for _ in range(split_index)] + [1 for _ in range(split_index)])

    xtest_data = x_second_label[split_index:len(x_first_label)] + x_first_label[split_index:len(x_first_label)]
    ytest = array([0 for _ in range(len(x_first_label)-split_index)] + [1 for _ in range(len(x_first_label)-split_index)])

    # fit the tokenizer on the documents
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x_first_label+x_second_label)

    # encode training data set
    Xtrain = tokenizer.texts_to_matrix(xtrain_data, mode=mode)

    #tokenizer.fit_on_texts(xtest_data)
    Xtest = tokenizer.texts_to_matrix(xtest_data, mode=mode)

    return Xtrain, ytrain, Xtest, ytest

In [ ]:
# There are many ways to define BOW: these are four of the most common ways

[Xtrain, ytrain, Xtest, ytest] = create_test_train(all_sents,'count',1000)
# [Xtrain, ytrain, Xtest, ytest] = create_test_train(1000, 'freq',first_label_df ,second_label_df)
# [Xtrain, ytrain, Xtest, ytest] = create_test_train(1000, 'binary',first_label_df ,second_label_df)
# [Xtrain, ytrain, Xtest, ytest] = create_test_train(1000, 'tfidf',first_label_df ,second_label_df)

### Create models: MultiLayer Perceptron

In [ ]:
# Define Network
n_words = Xtrain.shape[1]
model = Sequential()
model.add(Dense(50, input_shape=(n_words,), activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit network
model.fit(Xtrain, ytrain, epochs=50, verbose=2)

Epoch 1/50
2954/2954 - 3s - loss: 0.5127 - accuracy: 0.7478
Epoch 2/50
2954/2954 - 2s - loss: 0.4758 - accuracy: 0.7679
Epoch 3/50
2954/2954 - 2s - loss: 0.4597 - accuracy: 0.7789
Epoch 4/50
2954/2954 - 2s - loss: 0.4444 - accuracy: 0.7894
Epoch 5/50
2954/2954 - 2s - loss: 0.4281 - accuracy: 0.7999
Epoch 6/50
2954/2954 - 2s - loss: 0.4096 - accuracy: 0.8100
Epoch 7/50
2954/2954 - 2s - loss: 0.3898 - accuracy: 0.8223
Epoch 8/50
2954/2954 - 2s - loss: 0.3690 - accuracy: 0.8329
Epoch 9/50
2954/2954 - 2s - loss: 0.3476 - accuracy: 0.8446
Epoch 10/50
2954/2954 - 2s - loss: 0.3272 - accuracy: 0.8548
Epoch 11/50
2954/2954 - 2s - loss: 0.3087 - accuracy: 0.8635
Epoch 12/50
2954/2954 - 2s - loss: 0.2927 - accuracy: 0.8706
Epoch 13/50
2954/2954 - 2s - loss: 0.2784 - accuracy: 0.8768
Epoch 14/50
2954/2954 - 2s - loss: 0.2657 - accuracy: 0.8832
Epoch 15/50
2954/2954 - 2s - loss: 0.2549 - accuracy: 0.8876
Epoch 16/50
2954/2954 - 2s - loss: 0.2459 - accuracy: 0.8917
Epoch 17/50
2954/2954 - 2s - loss

In [ ]:
# evaluate mode Freq
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))


Test Accuracy: 67.699999


In [ ]:
# evaluate mode count
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))


Test Accuracy: 66.900003


In [ ]:
# evaluate mode binary
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))


Test Accuracy: 65.549999


In [ ]:
# evaluate mode tfidf
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))


Test Accuracy: 64.899999


### Logistic Regression

In [ ]:
Xtrain, ytrain, Xtest, ytest

In [ ]:
def simple_logistic_classify(Xtrain, ytrain, Xtest, ytest, description, _C=1.0):
    model = LogisticRegression(C=_C).fit(Xtrain, ytrain)
    score = model.score(Xtest, ytest)
    print('Test Score with', description, 'features', score)
    return model

In [ ]:
# evaluate mode Freq
model = simple_logistic_classify(Xtrain, ytrain, Xtest, ytest, 'frequency', _C=1.0)

Test Score with frequency features 0.6615


/cluster/tufts/sunterlab/coguik01/conda/condaenv/Emeka_conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
# evaluate mode binary
model = simple_logistic_classify(Xtrain, ytrain, Xtest, ytest, 'frequency', _C=1.0)

Test Score with frequency features 0.6855


/cluster/tufts/sunterlab/coguik01/conda/condaenv/Emeka_conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
# evaluate mode count
model = simple_logistic_classify(Xtrain, ytrain, Xtest, ytest, 'frequency', _C=1.0)

Test Score with frequency features 0.689


/cluster/tufts/sunterlab/coguik01/conda/condaenv/Emeka_conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
# evaluate mode tfidf
model = simple_logistic_classify(Xtrain, ytrain, Xtest, ytest, 'frequency', _C=1.0)

Test Score with frequency features 0.6915


/cluster/tufts/sunterlab/coguik01/conda/condaenv/Emeka_conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
# ideas gam; word2vec with words associaated with data in bow (include words associated with data to the verb features);